In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# basic_chat_langchain_gradio

> A basic gradio chat bot using OpenAI API's

In [ ]:
#| default_exp basic_chat_langchain_gradio

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| exporti
import jarvis.secrets
import jarvis.basic_chat_gradio as basic_chat_gradio

In [ ]:
#| exporti
import openai
import numpy as np
import os
import json
import gradio as gr
import datetime

In [ ]:
# from langchain.cache import SQLiteCache
# import langchain

In [ ]:
# langchain.llm_cache = SQLiteCache(database_path="../.langchain.db")

Setting the default environment API key for the OpenAI API python client.

In [ ]:
openai.api_key = os.environ["OPENAI_API_KEY"]

List of all OpenAI model listing the root model and release date, sorted by release date with the most recent at the top.

In [ ]:
# # | eval: false
# # | output: false
# demo = basic_chat_gradio.create_and_run_basic_ui()

In [ ]:
# #| eval: false
# #| output: false
# demo.close()

Closing server running on port: 7861


## Completion Messages

## Gradio Interface

`parse_codeblock` is a temporary fix for gradio displaying code correctly inside of triple backticks.

TODO: 
- Add tiktoken token counter for streaming api. https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
- Add running total calculator to track tokens and $.

In [ ]:
#|exports
def process_user_message(
    user_message:str, #Most recent message from user.
    history:list[list[str]] #Message history from gradio.Chatbot instance.
)->tuple[str,list[list[str]]]:
    """Add user message to 'history' and clear message textbox."""
    print('process_user_message',user_message,history)
    # if history is None: history = []
    return "", history + [[user_message, None]]

In [ ]:
#|exports
def process_bot_message(
    history:list[list[str]], #Message history from gradio.Chatbot instance.
    system_msg:str, #Most recent message from user.
    selected_model:str, #Selected OpenAI model for chat completion.
    temperature:float=0., #How deterministic the model results are.  See OpenAI docs.
    stream:bool=True, #Whether or not to stream the results or return all at once when the model has finished its response.
)->list[list[str]]:
    """Send chat history with most recent user message to the chat completions API."""
    stream_results = ''
    history[-1][1] = ''
    # try:
    response = openai.ChatCompletion.create(
        model=selected_model,
        messages=[{'role':'system', 'content': system_msg}] + gradio_chat_history_to_openai_chat_completions_format(history),
        temperature=temperature,
        stream=stream  # this time, we set stream=True
    )
    if stream:
        for chunk in response:
            model_results.append(model_results)
            # print(chunk)
            # print(chunk.get('choices',[{}])[0].get('delta',{}).get('content',''),end='')
            stream_results += chunk.get('choices',[{}])[0].get('delta',{}).get('content','')
            # history[-1][1] = parse_codeblock(stream_results) #removed because gradio properly formats code blocks now
            history[-1][1] = stream_results
            yield history
    else:
        # print(response)
        # print(response.get('choices',[{}])[0].get('message',{}).get('content',''))
        # print(parse_codeblock(response.get('choices',[{}])[0].get('message',{}).get('content','')))
        # history[-1][1] = parse_codeblock(response.get('choices',[{}])[0].get('message',{}).get('content','')) #gradio fixed code blocks natively
        history[-1][1] = response.get('choices',[{}])[0].get('message',{}).get('content','')
        # print(history)
        yield history
    # except:
    #     pass
    #     # print(history, system_msg, [{'role':'system', 'content': system_msg}] + history_to_chatgpt_format(history))

    # print(response)
    # print(stream_results)
    # print(parse_codeblock(stream_results))
    # print(history[-1][1])

In [ ]:
model_results = []
with gr.Blocks(title='Jarvis - My ChatGPT') as demo:
    with gr.Tab(label='Chat'):
        chatbot = gr.Chatbot()
        # chatbot.change(fn=lambda: None, scroll_to_output=True) #This also doesn't work. Autoscroll when chatbot in tab is known issue.
        with gr.Row():
            with gr.Column(scale=10):
                msg = gr.Textbox()
            with gr.Column(scale=1):
                clear = gr.Button("Clear")
                submit = gr.Button("Send")
    with gr.Tab(label='Settings'):
        system_msg = gr.Textbox(value='You are a helpful assistant.', info='This is the system message for OpenAI Chat APIs.')
        model_selection = gr.Dropdown(choices=['gpt-3.5-turbo','gpt-4'],value='gpt-3.5-turbo',label='Select model: ')
        temperature = gr.Slider(maximum=2., value=0., step=.1, label='Temperature', info='What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic.',)
        stream_results = gr.Checkbox(value=True, label='Stream Results', )

    submit0_kwargs = dict(fn=process_user_message, inputs=[msg, chatbot], outputs=[msg, chatbot])
    submit1_kwargs = dict(fn=process_bot_message, inputs=[chatbot, system_msg, model_selection, temperature, stream_results], outputs=chatbot)
    msg.submit(**submit0_kwargs).then(**submit1_kwargs)
    clear.click(fn=lambda: (None, None), inputs=None, outputs=[chatbot,msg], queue=False)
    submit.click(**submit0_kwargs).then(**submit1_kwargs)

In [ ]:
#| eval: false
#| output: false
demo.queue(concurrency_count=2)
demo.launch(server_name='0.0.0.0', inline=False)#,server_port=7860)

In [ ]:
#| eval: false
demo.close()

Closing server running on port: 7860


In [ ]:
#| eval: false
gr.close_all()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()